In [10]:
# set up an absolute path to the project 
# not needed in case of `pip install`
%run -i ../tools/setup_env.py

## Usage examples of `torchcnnbuilder.builder`

## Warning: The following examples contain a demonstration of custom implemented errors that a user may receive when working with the builder components. 

This submodule contains `Builder` class, which creates convolutional and transposed convolutional sequences

### `Builder`

In [3]:
from torchcnnbuilder.builder import Builder

Initialization params:

- **input_size** (Optional[Sequence[int]]): input size of the input tensor. Default: None
- **minimum_feature_map_size** (Sequence[int] | int): minimum feature map size. Default: 5
- **max_channels** (int): maximum number of layers after any convolution. Default: 512
- **min_channels** (int): minimum number of layers after any convolution. Default: 32
- **activation_function** (nn.Module): activation function. Default: nn.ReLU(inplace=True)
- **finish_activation_function** (Optional[nn.Module] | str): last activation function, can be same as activation_function (str `'same'`). Default: None

Other attributes:
- **conv_channels** (List[int]): list of output channels after each convolutional layer
- **transpose_conv_channels** (List[int]): list of output channels after each transposed convolutional layer
- **conv_layers** (List[tuple]): list of output tensor sizes after each convolutional layer
- **transpose_conv_layers** (List[tuple]): list of output tensor sizes after each transposed convolutional layer

Suppose we create a CNN-model for a tensor of size 100 by 100, let the smallest feature map after the convolutions be of size 3 by 3

In [4]:
builder = Builder(input_size=[125, 125], 
                  minimum_feature_map_size=3)

#### `Builder.build_convolve_sequence`

Params:

- **n_layers**: number of the convolution layers in the encoder part
- **in_channels**: number of channels in the first input tensor. Default: 1
- **params**: convolutional layer parameters (`nn.Conv2d`). Default: None
- **normalization**: choice of normalization between str `dropout`, `instancenorm` and `batchnorm`. Default: None
- **sub_blocks**: number of convolutions in one layer. Default: 1
- **p**: probability of an element to be zero-ed (for `dropout`). Default: 0.5
- **inplace**: if set to True, will do this operation in-place (for `dropout`). Default: False
- **eps**: a value added to the denominator for numerical stability (for `batchnorm`/`instancenorm`). Default: 1e-5
- **momentum**: used for the running_mean or running_var computation. Can be None for cumulative moving average (for `batchnorm`/`instancenorm`). Default: 0.1
- **affine**: a boolean value that when set to True, this module has learnable affine parameters (for `batchnorm`/`instancenorm`). Default: True
- **ratio**: multiplier for the geometric progression of increasing channels (feature maps). Used for `channel_growth_rate` as `exponential` or `power` . Default: 2 (powers of two)
- **start**: start position of a geometric progression in the case of `channel_growth_rate=exponential`. Default: 32
- **channel_growth_rate**: the way of calculating the number of feature maps between `exponential`, `proportion`, `power`, `linear` and `constant`. Default: `exponential`
- **conv_dim**: the dimension of the convolutional operation. Default: 2

Returns: nn.Sequential convolutional sequence

> WARNING: You cannot use this method without `input_size` param for the `Builder` initialization

The method helps to build convolutional sequences without writing the entire code by hand. The activation function is the one that was given at the time of initialization of the entire class. It is possible to use different types of normalization, but to manipulate the hyperparameters of this layer, it is necessary to build a convolutional sequence of convolutional blocks separately *(method `build_convolve_block`)*. The number of channels (feature maps) after each layer is calculated depending on the parameter `channel_growth_rate`:

$$
\begin{cases} 
    channel_i = start \times ratio^{i}, \quad i={1...n} & \text{if } \text{exponential} \\
    stop = \lfloor \displaystyle\frac{(input\_size[0] + input\_size[1]) * 0.5}{2} \rfloor + in\_channels & \text{if } \text{proportion} \\
    step = \displaystyle\frac{stop - in\_channels}{n\_layers} \\ 
    channels = \text{range}(in\_channels, stop, step) \\
    channel_i = in\_channels + i, \quad i={1...n}  & \text{if } \text{linear} \\
    channel_i = (in\_channels + i)^{ratio} \quad i={1...n} & \text{if } \text{power} \\
    channel_i = in\_channels, \quad i={1...n}  & \text{if } \text{constant}
\end{cases}
$$


In [5]:
builder.build_convolve_sequence(n_layers=3)

Sequential(
  (conv 1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (conv 2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (conv 3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
)

Sometimes the convolutional layer parameters you select can cause the tensor to degenerate after one of the layers or become smaller than the specified minimum size of the feature map. In this case, you will receive the corresponding error

In [6]:
builder.build_convolve_sequence(n_layers=5, 
                                params={'kernel_size': 25, 'padding': 1, 'dilation': 2})

ValueError: Input size and parameters can not provide more than 4 layers.

Normalization layers come with standard parameters as in `torch`. To change the parameters of convolutional layers, pass them as a dictionary

In [6]:
builder.build_convolve_sequence(n_layers=2, 
                                in_channels=3,
                                params={'kernel_size': 5, 'padding': 1})

Sequential(
  (conv 1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
  )
  (conv 2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
  )
)

In [7]:
builder.build_convolve_sequence(n_layers=2,
                                in_channels=3,
                                normalization='batchnorm',
                                eps=1e-3)

Sequential(
  (conv 1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv 2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
)

In [8]:
builder.build_convolve_sequence(n_layers=2,
                                in_channels=3,
                                normalization='dropout',
                                p=0.1)

Sequential(
  (conv 1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): Dropout2d(p=0.1, inplace=False)
    (2): ReLU(inplace=True)
  )
  (conv 2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): Dropout2d(p=0.1, inplace=False)
    (2): ReLU(inplace=True)
  )
)

You can build a convolutional sequence from convolutional blocks. Each convolutional block is convolutional layers one after another, passing through which the tensor does not change its size. While this is an early functional, there is no point in making a convolutional sequence of such blocks, since the tensor will not change its size after going through the entire sequence, for this it is necessary to add the implementation of pooling layers

In [9]:
builder.build_convolve_sequence(n_layers=2,
                                in_channels=3,
                                sub_blocks=2,
                                normalization='instancenorm')

Sequential(
  (conv 1): Sequential(
    (sub-block 1): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): ReLU(inplace=True)
    )
    (sub-block 2): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): ReLU(inplace=True)
    )
  )
  (conv 2): Sequential(
    (sub-block 1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): ReLU(inplace=True)
    )
    (sub-block 2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): ReLU(i

An example of using channel calculation using the `channel_growth_rate=proportion`

In [10]:
builder.build_convolve_sequence(n_layers=2,
                                in_channels=3,
                                params={'kernel_size': 9},
                                sub_blocks=2, 
                                channel_growth_rate='proportion')

Sequential(
  (conv 1): Sequential(
    (sub-block 1): Sequential(
      (0): Conv2d(3, 34, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
      (1): ReLU(inplace=True)
    )
    (sub-block 2): Sequential(
      (0): Conv2d(34, 34, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
      (1): ReLU(inplace=True)
    )
  )
  (conv 2): Sequential(
    (sub-block 1): Sequential(
      (0): Conv2d(34, 65, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
      (1): ReLU(inplace=True)
    )
    (sub-block 2): Sequential(
      (0): Conv2d(65, 65, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
      (1): ReLU(inplace=True)
    )
  )
)

To prevent the accidental creation of heavy models, by default there is a certain limit on the number of output channels (feature maps) after the convolutional layer. You can set it when initializing the class (or change it later as an attribute of the class)

In [7]:
builder.build_convolve_sequence(n_layers=5,
                                params={'kernel_size': 9},
                                in_channels=3,
                                channel_growth_rate='exponential', # by default
                                ratio=3)

ValueError: There is too many channels [[864]]. Max channels 512 [layer 4].

If you reduce the number of layers in the previous case, then there will be no error

In [8]:
builder.build_convolve_sequence(n_layers=3,
                                params={'kernel_size': 9},
                                in_channels=3,
                                channel_growth_rate='power',
                                ratio=3)

Sequential(
  (conv 1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(9, 9), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (conv 2): Sequential(
    (0): Conv2d(64, 125, kernel_size=(9, 9), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (conv 3): Sequential(
    (0): Conv2d(125, 216, kernel_size=(9, 9), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
)

Also, you can do the same by using 1d or 3d convolution layers, however `Builder` should get the same number of dimensions in the `input_size` *(or not less than `len(input_size) - conv_dim`)*. Otherwise, the corresponding error will be thrown. All normalization layers would be replaced with the corresponding N-dimensional methods

In [13]:
test_builder = Builder(input_size=(31,))
test_builder.build_convolve_sequence(n_layers=5,
                                    in_channels=1,
                                    normalization='dropout',
                                    conv_dim=1)

Sequential(
  (conv 1): Sequential(
    (0): Conv1d(1, 32, kernel_size=(3,), stride=(1,))
    (1): Dropout1d(p=0.5, inplace=False)
    (2): ReLU(inplace=True)
  )
  (conv 2): Sequential(
    (0): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
    (1): Dropout1d(p=0.5, inplace=False)
    (2): ReLU(inplace=True)
  )
  (conv 3): Sequential(
    (0): Conv1d(64, 128, kernel_size=(3,), stride=(1,))
    (1): Dropout1d(p=0.5, inplace=False)
    (2): ReLU(inplace=True)
  )
  (conv 4): Sequential(
    (0): Conv1d(128, 256, kernel_size=(3,), stride=(1,))
    (1): Dropout1d(p=0.5, inplace=False)
    (2): ReLU(inplace=True)
  )
  (conv 5): Sequential(
    (0): Conv1d(256, 512, kernel_size=(3,), stride=(1,))
    (1): Dropout1d(p=0.5, inplace=False)
    (2): ReLU(inplace=True)
  )
)

In [14]:
error_test_builder = Builder(input_size=(31,))
error_test_builder.build_convolve_sequence(n_layers=5,
                                           in_channels=1,
                                           normalization='instancenorm',
                                           conv_dim=3)

ValueError: The difference in dimensions between input_size input_size=(31,) and convolution conv_dim=3 should not be more than 1 (input_size.shape - conv_dim should be equal to 1 or 0)

#### `Builder.build_convolve_block`

Params:

- **in_channels**: number of channels in the input image
- **out_channels**: number of channels produced by the convolution
- **params**: convolutional layer parameters (`nn.Conv2d`). Default: None
- **normalization**: choice of normalization between str `dropout`, `instancenorm` and `batchnorm`. Default: None
- **sub_blocks**: number of convolutions in one layer. Default: 1
- **p**: probability of an element to be zero-ed (for `dropout`). Default: 0.5
- **inplace**: if set to True, will do this operation in-place (for `dropout`). Default: False
- **eps**: a value added to the denominator for numerical stability (for `batchnorm`/`instancenorm`). Default: 1e-5
- **momentum**: used for the running_mean or running_var computation. Can be None for cumulative moving average (for `batchnorm`/`instancenorm`). Default: 0.1
- **affine**: a boolean value that when set to True, this module has learnable affine parameters (for `batchnorm`/`instancenorm`). Default: True
- **conv_dim**: the dimension of the convolutional operation. Default: 2

Returns: nn.Sequential one convolution block with an activation function

If you want a more subtle selection of the hyperparameters of the convolutional layers, it is better to use the logic of convolutional blocks. In each such block, you can configure convolutions and normalization layers *(by default, all parameters are as in `torch`)*, and then combine these blocks into `nn.Sequential`

In [15]:
import torch
input_image = torch.rand(1, 3, 100, 100)

In [16]:
conv_layer = builder.build_convolve_block(in_channels=3, 
                                          out_channels=64, 
                                          normalization='dropout',
                                          p=0.2,
                                          sub_blocks=3)
conv_layer

Sequential(
  (sub-block 1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Dropout2d(p=0.2, inplace=False)
    (2): ReLU(inplace=True)
  )
  (sub-block 2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Dropout2d(p=0.2, inplace=False)
    (2): ReLU(inplace=True)
  )
  (sub-block 3): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Dropout2d(p=0.2, inplace=False)
    (2): ReLU(inplace=True)
  )
)

In [17]:
# checking that the tensor passes through the block
output = conv_layer(input_image)
output.shape

torch.Size([1, 64, 100, 100])

In [18]:
conv_layer = builder.build_convolve_block(in_channels=3, 
                                          out_channels=64, 
                                          params={'kernel_size': (7, 7), 'dilation': (3, 3)},
                                          normalization='instancenorm',
                                          eps=1e-7)
conv_layer

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), dilation=(3, 3))
  (1): InstanceNorm2d(64, eps=1e-07, momentum=0.1, affine=True, track_running_stats=False)
  (2): ReLU(inplace=True)
)

In [19]:
# checking that the tensor passes through the block
output = conv_layer(input_image)
output.shape

torch.Size([1, 64, 82, 82])

You can also change the dimension of block layers by using `conv_dim`

In [20]:
builder.build_convolve_block(in_channels=3,
                             out_channels=4,
                             normalization='dropout',
                             conv_dim=3)

Sequential(
  (0): Conv3d(3, 4, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (1): Dropout3d(p=0.5, inplace=False)
  (2): ReLU(inplace=True)
)

#### `Builder.build_transpose_convolve_sequence`

Params: 

- **n_layers**: number of the convolution layers in the encoder part
- **in_channels**: number of channels in the first input tensor. Default: 1
- **out_channels**: number of channels after the transposed convolution sequence. Default: 1
- **out_size**: output size after the transposed convolution sequence. Default: None (input size)
- **params**: convolutional layer parameters (`nn.Conv2d`). Default: None
- **normalization**: choice of normalization between str `dropout`, `instancenorm` and `batchnorm`. Default: None
- **sub_blocks**: number of convolutions in one layer. Default: 1
- **p**: probability of an element to be zero-ed (for `dropout`). Default: 0.5
- **inplace**: if set to True, will do this operation in-place (for `dropout`). Default: False
- **eps**: a value added to the denominator for numerical stability (for `batchnorm`/`instancenorm`). Default: 1e-5
- **momentum**: used for the running_mean or running_var computation. Can be None for cumulative moving average (for `batchnorm`/`instancenorm`). Default: 0.1
- **affine**: a boolean value that when set to True, this module has learnable affine parameters (for `batchnorm`/`instancenorm`). Default: True
- **ratio**: multiplier for the geometric progression of increasing channels (feature maps). Used for `channel_growth_rate` as `exponential` or `power` . Default: 2 (powers of two)
- **start**: start position of a geometric progression in the case of `channel_growth_rate=exponential`. Default: 32
- **channel_growth_rate**: the way of calculating the number of feature maps between `exponential`, `proportion`, `power`, `linear` and `constant`. Default: `exponential`
- **conv_dim**: the dimension of the convolutional operation. Default: 2
- **adaptive_pool**: choice of a last layer as an adaptive pooling between str `avgpool` or `maxpool`. Default: `avgpool`

Returns: nn.Sequential transposed convolutional sequence

The method helps to build transposed convolutional sequences without writing the entire code by hand. The activation function is the one that was given at the time of initialization of the entire class. It is possible to use different types of normalization, but to manipulate the hyperparameters of this layer, it is necessary to build a transposed convolutional sequence of transposed convolutional blocks separately *(method `build_transpose_convolve_block`)*. The number of channels (feature maps) after each layer is calculated depending on the parameter `channel_growth_rate`. See the calculation formulas above in the function, everything is considered the same here, but in the opposite direction. If an activation function was specified during initialization of the class *(see attributes during initialization)* after the last layer, then it will be applied

To eliminate the problem of size discrepancy between the output tensor after the decoder part and the input tensor before the encoder part *(due to the parity/dishonesty of the convolution parameters, some pixels may be lost)*, there is a pooling layer at the end

> WARNING: You cannot use this method without `input_size` param for the `Builder` initialization

In [21]:
builder.build_transpose_convolve_sequence(n_layers=3)

Sequential(
  (deconv 1): Sequential(
    (0): ConvTranspose2d(216, 108, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (deconv 2): Sequential(
    (0): ConvTranspose2d(108, 54, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (deconv 3): Sequential(
    (0): ConvTranspose2d(54, 1, kernel_size=(3, 3), stride=(1, 1))
  )
  (resize): AdaptiveAvgPool2d(output_size=(125, 125))
)

In [22]:
builder.build_transpose_convolve_sequence(n_layers=2, 
                                          in_channels=3,
                                          params={'kernel_size': 5, 'padding': 1})

Sequential(
  (deconv 1): Sequential(
    (0): ConvTranspose2d(3, 1, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
  )
  (deconv 2): Sequential(
    (0): ConvTranspose2d(1, 1, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  )
  (resize): AdaptiveAvgPool2d(output_size=(125, 125))
)

In [23]:
builder.build_transpose_convolve_sequence(n_layers=2,
                                          in_channels=10,
                                          out_channels=3,
                                          out_size=(130, 130),
                                          normalization='dropout',
                                          p=0.3,
                                          channel_growth_rate='linear')

Sequential(
  (deconv 1): Sequential(
    (0): ConvTranspose2d(10, 9, kernel_size=(3, 3), stride=(1, 1))
    (1): Dropout2d(p=0.3, inplace=False)
    (2): ReLU(inplace=True)
  )
  (deconv 2): Sequential(
    (0): ConvTranspose2d(9, 3, kernel_size=(3, 3), stride=(1, 1))
    (1): Dropout2d(p=0.3, inplace=False)
  )
  (resize): AdaptiveAvgPool2d(output_size=(130, 130))
)

Sometimes the transposed convolutional layer parameters you select can cause the number of feature maps to degenerate after one of the layers or become smaller than the specified minimum of channels. In this case, you will receive the corresponding error

In [24]:
builder.build_transpose_convolve_sequence(n_layers=6,
                                          in_channels=20,
                                          ratio=3,
                                          channel_growth_rate='exponential') # by default

/Users/kdduha/anaconda3/envs/TorchCNNBuilder/lib/python3.10/site-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


ValueError: There is too few channels [0]. You can not provide less then 1 channel [layer 3]

Examples of different final activation functions

In [25]:
import torch.nn as nn

# redefining the last activation function
builder.finish_activation_function = nn.Softmax()

builder.build_transpose_convolve_sequence(n_layers=3,
                                          in_channels=20,
                                          out_channels=3,
                                          normalization='dropout')

Sequential(
  (deconv 1): Sequential(
    (0): ConvTranspose2d(20, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): Dropout2d(p=0.5, inplace=False)
    (2): ReLU(inplace=True)
  )
  (deconv 2): Sequential(
    (0): ConvTranspose2d(10, 5, kernel_size=(3, 3), stride=(1, 1))
    (1): Dropout2d(p=0.5, inplace=False)
    (2): ReLU(inplace=True)
  )
  (deconv 3): Sequential(
    (0): ConvTranspose2d(5, 3, kernel_size=(3, 3), stride=(1, 1))
    (1): Dropout2d(p=0.5, inplace=False)
    (2): Softmax(dim=None)
  )
  (resize): AdaptiveAvgPool2d(output_size=(125, 125))
)

In [26]:
# redefining the last activation function
builder.finish_activation_function = 'same'

builder.build_transpose_convolve_sequence(n_layers=3,
                                          in_channels=30,
                                          out_channels=2,
                                          normalization='batchnorm',
                                          adaptive_pool='maxpool')

Sequential(
  (deconv 1): Sequential(
    (0): ConvTranspose2d(30, 15, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (deconv 2): Sequential(
    (0): ConvTranspose2d(15, 7, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (deconv 3): Sequential(
    (0): ConvTranspose2d(7, 2, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (resize): AdaptiveMaxPool2d(output_size=(125, 125))
)

#### `Builder.build_transpose_convolve_block`

Params:

- **in_channels**: number of channels in the input image
- **out_channels**: number of channels produced by the convolution
- **params**: convolutional layer parameters (nn.Conv2d). Default: None
- **normalization**: choice of normalization between str `dropout`, `instancenorm` and `batchnorm`. Default: None
- **sub_blocks**: number of convolutions in one layer. Default: 1
- **p**: probability of an element to be zero-ed. Default (for `dropout`): 0.5
- **inplace**: if set to True, will do this operation in-place. Default (for `dropout`): False
- **eps**: a value added to the denominator for numerical stability (for `batchnorm`/`instancenorm`). Default: 1e-5
- **momentum**: used for the running_mean or running_var computation. Can be None for cumulative moving average (for `batchnorm`/`instancenorm`). Default: 0.1
- **affine**: a boolean value that when set to True, this module has learnable affine parameters (for `batchnorm`/`instancenorm`). Default: True
- **last_block**: if True there is no activation function after the transposed convolution. Default: False
- **conv_dim**: the dimension of the convolutional operation. Default: 2

Returns: nn.Sequential one convolution block with an activation function

If you want a more subtle selection of the hyperparameters of the transposed convolutional layers, it is better to use the logic of transposed convolutional blocks. In each such block, you can configure transposed convolutions and normalization layers *(by default, all parameters are as in `torch`)*, and then combine these blocks into `nn.Sequential`

In [27]:
builder.finish_activation_function = nn.Softmax()
deconv_layer = builder.build_transpose_convolve_block(in_channels=3, 
                                                      out_channels=64, 
                                                      normalization='dropout',
                                                      p=0.2,
                                                      sub_blocks=3,
                                                      last_block=True)
deconv_layer

Sequential(
  (transpose sub-block 1): Sequential(
    (0): ConvTranspose2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Dropout2d(p=0.2, inplace=False)
    (2): ReLU(inplace=True)
  )
  (transpose sub-block 2): Sequential(
    (0): ConvTranspose2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Dropout2d(p=0.2, inplace=False)
    (2): ReLU(inplace=True)
  )
  (transpose sub-block 3): Sequential(
    (0): ConvTranspose2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Dropout2d(p=0.2, inplace=False)
    (2): Softmax(dim=None)
  )
)

In [28]:
deconv_layer = builder.build_transpose_convolve_block(in_channels=64, 
                                                      out_channels=3,
                                                      normalization='batchnorm',
                                                      eps=1e-4)
deconv_layer

Sequential(
  (0): ConvTranspose2d(64, 3, kernel_size=(3, 3), stride=(1, 1))
  (1): BatchNorm2d(3, eps=0.0001, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
)

#### Attributes examples and blocks comparison

Let's compare a convolutional block and a transposed convolutional block

In [29]:
conv_layer = builder.build_convolve_block(in_channels=3,
                                          out_channels=64,
                                          normalization='batchnorm',
                                          eps=1e-4)
conv_layer

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (1): BatchNorm2d(64, eps=0.0001, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
)

In [30]:
input_image = torch.rand(1, 3, 125, 125)

In [31]:
after_conv = conv_layer(input_image)
after_conv.shape

torch.Size([1, 64, 123, 123])

In [32]:
after_deconv = deconv_layer(after_conv)
after_deconv.shape

torch.Size([1, 3, 125, 125])

As we can see, the dimensions of the input and final output tensor are the same. Moreover, you can check the history of tensor sizes after conv and transposed conv sequences (as channels history)

In [33]:
print(f'Conv channels history: {builder.conv_channels}',
      f'Transposed channels history: {builder.transpose_conv_channels}',
      f'Conv sizes history: {builder.conv_layers}',
      f'Transposed sizes history: {builder.transpose_conv_layers}',
      sep='\n')

Conv channels history: [3, 64, 125, 216]
Transposed channels history: [30, 15, 7, 2]
Conv sizes history: [(125, 125), (123, 123), (121, 121), (119, 119)]
Transposed sizes history: [(119, 119), (121, 121), (123, 123), (125, 125)]
